In [1]:
###########################################
# IMPORTS
###########################################
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory, RESTTopologyFactoryV2
from perses.annihilation.lambda_protocol import RESTState, RESTStateV2
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")

conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.3 jupyter extensions


In [2]:
def compare_energies(topology, REST_system, other_system, positions, rest_atoms, T_min, T):

    # Get solvent atoms
    positive_ion_name = "NA"
    negative_ion_name = "CL"
    water_name = "HOH"
    solvent_atoms = []
    if 'openmm' in topology.__module__:
        atoms = topology.atoms()
    elif 'mdtraj' in topology.__module__:
        atoms = topology.atoms
    for atom in atoms:
        if atom.residue.name == positive_ion_name or atom.residue.name == negative_ion_name or atom.residue.name == water_name:
            solvent_atoms.append(atom.index)
    
    # Create thermodynamic state
    lambda_zero_alchemical_state = RESTStateV2.from_system(REST_system)
    thermostate = ThermodynamicState(REST_system, temperature=T_min)
    compound_thermodynamic_state = CompoundThermodynamicState(thermostate,
                                                              composable_states=[lambda_zero_alchemical_state])
    
    # Set alchemical parameters
    beta_0 = 1 / (kB * T_min)
    beta_m = 1 / (kB * T)
    compound_thermodynamic_state.set_alchemical_parameters(beta_0, beta_m)
    print("beta_0: ", beta_0)
    print("beta_m: ", beta_m)
    
    # Minimize and save energy
    integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context = compound_thermodynamic_state.create_context(integrator)
    context.setPositions(positions)
    sampler_state = SamplerState.from_context(context)
    REST_energy = compound_thermodynamic_state.reduced_potential(sampler_state)
    
    # Get energy components for rest system
    components_rest = [component for component in compute_potential_components(context, beta=beta)]
    print(components_rest)
    
    # Compute energy for non-RESTified system
    # Determine regions and scaling factors
    nonrest_atoms = [i for i in range(other_system.getNumParticles()) if i not in rest_atoms and i not in solvent_atoms]
    rest_scaling = beta_m / beta_0
    inter_scaling = np.sqrt(beta_m / beta_0)

    # Scale the terms in the bond force appropriately
    bond_force = other_system.getForce(0)
    for bond in range(bond_force.getNumBonds()):
        p1, p2, length, k = bond_force.getBondParameters(bond)
        if p1 in rest_atoms and p2 in rest_atoms:
            bond_force.setBondParameters(bond, p1, p2, length, k * rest_scaling)
        elif (p1 in rest_atoms and p2 in nonrest_atoms) or (p1 in nonrest_atoms and p2 in rest_atoms):
            bond_force.setBondParameters(bond, p1, p2, length, k * inter_scaling)

    # Scale the terms in the angle force appropriately
    angle_force = other_system.getForce(1)
    for angle_index in range(angle_force.getNumAngles()):
        p1, p2, p3, angle, k = angle_force.getAngleParameters(angle_index)
        if p1 in rest_atoms and p2 in rest_atoms and p3 in rest_atoms:
            angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * rest_scaling)
        elif set([p1, p2, p3]).intersection(set(rest_atoms)) != set() and set([p1, p2, p3]).intersection(set(nonrest_atoms)) != set():
            angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * inter_scaling)

    # Scale the terms in the torsion force appropriately
    torsion_force = other_system.getForce(2)
    for torsion_index in range(torsion_force.getNumTorsions()):
        p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
        if p1 in rest_atoms and p2 in rest_atoms and p3 in rest_atoms and p4 in rest_atoms:
            torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * rest_scaling)
        elif set([p1, p2, p3, p4]).intersection(set(rest_atoms)) != set() and set([p1, p2, p3, p4]).intersection(set(nonrest_atoms)) != set():
            torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * inter_scaling)

    # Scale the exceptions in the nonbonded force appropriately
    nb_force = other_system.getForce(3)
    for nb_index in range(nb_force.getNumExceptions()):
        p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(nb_index)
        if (p1 in rest_atoms and p2 in rest_atoms):
            nb_force.setExceptionParameters(nb_index, p1, p2, rest_scaling * chargeProd, sigma, rest_scaling * epsilon)
        elif (p1 in rest_atoms and p2 in nonrest_atoms) or (p1 in nonrest_atoms and p2 in rest_atoms):
            nb_force.setExceptionParameters(nb_index, p1, p2, inter_scaling * chargeProd, sigma, inter_scaling * epsilon)

    # Scale nonbonded interactions for rest-rest region by adding exceptions for all pairs of atoms
    exception_pairs = [tuple(sorted([nb_force.getExceptionParameters(nb_index)[0], nb_force.getExceptionParameters(nb_index)[1]])) for nb_index in range(nb_force.getNumExceptions())]
    rest_pairs = set([tuple(sorted(pair)) for pair in list(itertools.product(rest_atoms, rest_atoms))])
    for pair in list(rest_pairs):
        p1 = pair[0]
        p2 = pair[1]
        p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
        p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
        if p1 != p2:
            if pair not in exception_pairs:
                nb_force.addException(p1, p2, p1_charge * p2_charge * rest_scaling, 0.5 * (p1_sigma + p2_sigma),
                                      np.sqrt(p1_epsilon * p2_epsilon) * rest_scaling)

    # Scale nonbonded interactions for inter region by adding exceptions for all pairs of atoms
    for pair in list(itertools.product(rest_atoms, nonrest_atoms)):
        p1 = pair[0]
        p2 = int(pair[1])  # otherwise, will be a numpy int
        p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
        p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
        if tuple(sorted(pair)) not in exception_pairs:
            nb_force.addException(p1, p2, p1_charge * p2_charge * inter_scaling, 0.5 * (p1_sigma + p2_sigma), np.sqrt(p1_epsilon * p2_epsilon) * inter_scaling)

    # Scale nonbonded interactions for rest-water region by adding exceptions for all pairs of atoms
    for pair in list(itertools.product(rest_atoms, solvent_atoms)):
        p1 = pair[0]
        p2 = pair[1]
        p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
        p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
        nb_force.addException(p1, p2, p1_charge * p2_charge * rest_scaling, 0.5 * (p1_sigma + p2_sigma), np.sqrt(p1_epsilon * p2_epsilon) * rest_scaling)
            
    # Get energy
    thermostate = ThermodynamicState(other_system, temperature=T_min)
    integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context = thermostate.create_context(integrator)
    context.setPositions(positions)
    sampler_state = SamplerState.from_context(context)
    nonREST_energy = thermostate.reduced_potential(sampler_state)
    
    print(f"Energies: {REST_energy} (rest), {nonREST_energy} (nonrest)")
    print(f"Discrepancy: {abs(REST_energy - nonREST_energy)}")
    assert abs(REST_energy - nonREST_energy) < 1, f"The energy of the REST system ({REST_energy}) does not match " \
                                                        f"that of the non-REST system with terms manually scaled according to REST2({nonREST_energy})."
    
    print("Success!")


In [16]:
def test_energy_scaling():
    """
        Test whether the energy of a REST-ified system is equal to the energy of the system with terms manually scaled by
        the same factor as is used in REST.  T_min is 298 K and the thermodynamic state has temperature 600 K.
    """

    # Set temperatures
    T_min = 298.0 * unit.kelvin
    T = 600 * unit.kelvin

    ## CASE 1: alanine dipeptide in vacuum
    # Create vanilla system
    ala = AlanineDipeptideVacuum()
    system = ala.system
    system.removeForce(4)
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV2(system, topology, rest_region=rest_atoms)
    REST_system = factory.REST_system

    # Check energy scaling
    compare_energies(topology, REST_system, system, positions, rest_atoms, T_min, T)

    ## CASE 2: alanine dipeptide in solvent
    # Create vanilla system
    ala = AlanineDipeptideExplicit()
    system = ala.system
    system.removeForce(4)
    system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV2(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system
    REST_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
    REST_system.getForce(4).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)


    # Check energy scaling
    compare_energies(topology, REST_system, system, positions, rest_atoms, T_min, T)

    ## CASE 3: alanine dipeptide in solvent with repartitioned hybrid system
    # Create repartitioned hybrid system for lambda 0 endstate
    atp, system_generator = generate_atp(phase='solvent')
    htf = generate_dipeptide_top_pos_sys(atp.topology,
                                         new_res='THR',
                                         system=atp.system,
                                         positions=atp.positions,
                                         system_generator=system_generator,
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=0,
                                         validate_endstate_energy=False)
    system = htf.hybrid_system
    system.removeForce(0) # Remove barostat
    system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)

    # Create REST-ified hybrid system
    res1 = list(htf.hybrid_topology.residues)[1]
    rest_atoms = [atom.index for atom in list(res1.atoms)]
    factory = RESTTopologyFactoryV2(system, htf.hybrid_topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system
    REST_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
    REST_system.getForce(4).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)

    # Check energy scaling
    compare_energies(htf.hybrid_topology, REST_system, system, htf.hybrid_positions, rest_atoms, T_min, T)

In [17]:
test_energy_scaling()

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


solvent atoms: []
beta_0:  0.00040359850685478543 mol/J
beta_m:  0.00020045392507121008 mol/J
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.034289926452607325), ('CustomAngleForce', 0.3540266924808168), ('CustomTorsionForce', 2.275322602508427), ('NonbondedForce', -39.186929599617905), ('CustomNonbondedForce', 47.4462242032202), ('CustomBondForce', -49.27626333837166), ('AndersenThermostat', 0.0)]


INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]


Energies: -38.61073440939011 (rest), -38.61074063166303 (nonrest)
Discrepancy: 6.222272915579197e-06
Success!


INFO:REST:No unknown forces.


...

INFO:geometry:total reduced energy added from growth system: 20.672986672125617
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467813837657
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -26805.059322491197


conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463328533253), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RepartitionedHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.56477354), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:rel

solvent atoms: [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 23

conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.2725643826330057), ('CustomAngleForce', 5.729163090687007), ('CustomTorsionForce', 23.07854953733333), ('NonbondedForce', -6555.920840710024), ('CustomNonbondedForce', 51.18105657620016), ('CustomBondForce', -9.213558385610364), ('AndersenThermostat', 0.0)]
Energies: -6528.395703532329 (rest), -6528.3957084488175 (nonrest)
Discrepancy: 4.916488251183182e-06
Success!


In [31]:
compound_thermostate.rest_scale

0.49666666666666665

In [32]:
compound_thermostate.inter_scale

0.7047458170621992

In [33]:
compound_thermostate.rest_nb_scale

-0.5033333333333334

In [34]:
compound_thermostate.inter_nb_scale

-0.2952541829378008

In [17]:
REST_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6d192c8cf0> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b6d192c8cc0> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b6d192c8180> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b6d192c85d0> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2b6d192c8fc0> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6d29a924b0> >]

In [3]:
## CASE 2: alanine dipeptide in solvent
# Create vanilla system
ala = AlanineDipeptideExplicit()
system = ala.system
positions = ala.positions
topology = ala.topology

# Create REST system
system.removeForce(4)
# system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = RESTTopologyFactoryV2(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
REST_system = factory.REST_system
# REST_system.removeForce(5)
# REST_system.removeForce(4)
# REST_system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)
# REST_system.getForce(4).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)

# REST_system.getForce(4).setUseSwitchingFunction(True)
# REST_system.getForce(4).setUseLongRangeCorrection(False)

# Set temperatures
T_min = 298.0 * unit.kelvin
T = 600 * unit.kelvin

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]
INFO:REST:No unknown forces.
INFO:REST:Handling constraints
INFO:REST:Handling bonds
INFO:REST:Handling angles
INFO:REST:Handling torsions
INFO:REST:Handling nonbondeds
INFO:REST:Handling nonbonded scaling (custom nb)
INFO:REST:Handling nonbonded exception scaling (custom bond)


In [4]:
system.getForce(3).getNonbondedMethod()

4

In [5]:
openmm.NonbondedForce.PME

4

In [13]:
openmm.NonbondedForce.CutoffPeriodic

2

In [6]:
system.getForce(3).getUseDispersionCorrection() 

True

In [7]:
REST_system.getForce(3).getUseSwitchingFunction()

True

In [11]:
REST_system.getForce(3).getNonbondedMethod()

4

In [12]:
REST_system.getForce(4).getNonbondedMethod()

2

In [15]:
REST_system.getForce(4).getUseLongRangeCorrection() 

False

In [8]:
REST_system.getForce(3).getSwitchingDistance()

Quantity(value=0.8500000000000001, unit=nanometer)

In [9]:
compare_energies(topology, REST_system, system, positions, rest_atoms, T_min, T)

beta_0:  0.00040359850685478543 mol/J
beta_m:  0.00020045392507121008 mol/J
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.03428993046066216), ('CustomAngleForce', 0.3540264365714315), ('CustomTorsionForce', 2.2753213990081447), ('NonbondedForce', -9889.978580573847), ('CustomNonbondedForce', 31.52612867663931), ('CustomBondForce', -49.27626380347902), ('AndersenThermostat', 0.0)]
Energies: -9971.529842339349 (rest), -9946.698136942086 (nonrest)
Discrepancy: 24.83170539726234


AssertionError: The energy of the REST system (-9971.529842339349) does not match that of the non-REST system with terms manually scaled according to REST2(-9946.698136942086).